In [1]:
import pandas as pd
import datetime
import pyupbit

import math
import time

In [11]:
# utils

def make_levels(mode, upper, lower, grids):
    # grid level 생성
    lower = pyupbit.get_tick_size(lower)
    upper = pyupbit.get_tick_size(upper)
    levels = [lower]
    cnt = 1
    # 동일한 가격 차이로 levels 생성
    if mode == "Arithmetic":
        while cnt < grids:
            diff =  (upper - lower)/(grids) # 주문들 사이의 가격 차이
            price = lower + diff * cnt
            price = pyupbit.get_tick_size(price) # 업비트 호가에 맞춰서 조정
            levels.append(price)
            cnt += 1
    # 동일한 가격 비율로 levels 생성
    elif mode == "Geometric":
        while cnt < grids:
            diff = (upper / lower) ** (1/(grids)) # 주문들 사이의 가격 비율 차이
            price = lower * (diff ** cnt)
            price = pyupbit.get_tick_size(price) # 업비트 호가에 맞춰서 조정
            levels.append(price)
            cnt += 1
            
    # # 마지막에 UPPER 값 넣기
    # levels.append(upper)
    
    return levels, diff

# 호가 단위 함수
def get_price_scale_tick(_price):   
    if _price >= 2000000: 
        return -3, 1000
    elif _price >= 1000000: 
        return -2, 500
    elif _price >= 500000: 
        return -2, 100
    elif _price >= 100000: 
        return -1, 50
    elif _price >= 10000: 
        return -1, 10
    elif _price >= 1000: 
        return 0, 5
    elif _price >= 100: 
        return 0, 1
    elif _price >= 10: 
        return 1, 0.1
    elif _price >= 0: 
        return 2, 0.01
    
# 미체결 주문 모두 취소
def cancel_all_order(coin):
    order_list = upbit.get_order(f"KRW-{coin}")
    for order in order_list:
        ret_cancel = upbit.cancel_order(order["uuid"])
        if 'error' in ret_cancel.keys():
            print(ret_cancel['error']['message'])
        time.sleep(0.1)
        
# 코인 보유량 모두 시장가 매도
def sell_all_market(coin):
    share = upbit.get_balance(coin)
    ret_stoploss = upbit.sell_market_order(f"KRW-{coin}", share)
    # 에러 발생시 message 출력
    if 'error' in ret_stoploss.keys():
        print(ret_stoploss['error']['message'])
        
# 잔고 중 해당하는 코인의 평단가 불러오기
def get_profit(coin):
    balances_list = upbit.get_balances()
    for balances in balances_list:
        if balances["currency"] == coin: 
            avg_price = float(balances["avg_buy_price"])
            break
    # 현재 가격과 비교해서 수익률 계산
    current = pyupbit.get_current_price(f"KRW-{coin}")
    profit = (current-avg_price)/avg_price*100
    return current, avg_price, profit

# 과거 데이터 조회해서 변동성 계산
def get_std(coin, interval, std_num):
    df = pyupbit.get_ohlcv(f"KRW-{coin}", interval=f"minute{interval}")
    std = df.iloc[-std_num:].close.std()
    return std

In [12]:
# upbit 연결
with open("upbit_key.txt", 'r') as f:
    lines = f.read().split('\n')
    access_key = lines[0]
    secret_key = lines[1]
upbit = pyupbit.Upbit(access_key, secret_key)

In [21]:
# 지정 파라미터
MODE = "Geometric" # Arithmetic: 동일한 가격 차이 / Geometric: 동일한 가격 비율 차이
GRIDS = 5 # 그리드 수
COIN = "BTC" # 거래할 코인
BUDGET = 80000 # 사용할 예산
INTERVAL = 30 # 변동성 계산할 때 몇분 봉 사용
STD_NUM = 40 # 변동성 계산할 때 과거 몇개 사용
LOWER_STD = 3 # 하한선 변동성 배수

STOP_LOSS = 10 # 손절매 퍼센트
RESET_GRID = 5 # 계속 상승할 경우 그리드 리셋하는 퍼센트

In [62]:
# 가격이 일정 기준 이상 오르면 GRID 리셋
current = pyupbit.get_current_price(f"KRW-{COIN}")
if (current-levels[-1])/levels[-1]*100 > RESET_GRID:
    
    
    levels, diff, volume = get_evn(COIN, MODE, GRIDS, BUDGET, INTERVAL, STD_NUM, LOWER_STD)
    # RESET

27087000

In [29]:
def get_evn(coin, mode, grids, budget, interval, std_num, lower_std):
    # 현재가 조회
    current = pyupbit.get_current_price(f"KRW-{coin}")

    # 잔고 조회
    balance_KRW = upbit.get_balance("KRW")
    balance_coin = upbit.get_balance(coin)
    if balance_coin != 0:
        raise Exception("코인 수가 0이 아닙니다.")

    print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    print()
    print(f"{coin} 현재가(KRW):",current)

    # 호가 단위
    _, scale = get_price_scale_tick(current)
    print("호가단위(KRW):", scale)

    # 변동성 계산
    std = get_std(coin, interval, std_num)
    print(f"{interval}분봉 {std_num}개 변동성: {std:.2f}")
    print(f"Lower Std: {LOWER_STD}")

    # 상한선 하한선 계산
    upper = current 
    lower = current - lower_std*std
    if current < upper:
        raise Exception("상한선은 현재가보다 낮아야합니다.")

    # grid levels 생성
    levels, diff = make_levels(mode, upper, lower, grids)
    print("Levels:", levels)
    if mode == "Arithmetic":
        print(f"Difference(KRW): {diff:.4f}")
    else:
        print(f"Difference(%): {(diff-1)*100:.2f}%")

    # 주문량 생성
    volume = math.floor((budget / GRIDS)/current*1e8)/1e8 # 업비트는 소수 8자리까지 주문 가능
    if volume * current < 5000: # 최소 주문 단위 5000보다 작을 경우 error
        raise Exception(f"최소 주문 단위보다 작습니다. {volume * current:.2f}원")
    print(f"Volume({coin}):", volume)
    
    if mode == "Arithmetic":
        print(f"수익률: {diff/current*100:.2f}%")
    else:
        print(f"수익률: {(diff-1)*100:.2f}%")
    
    return levels, diff, volume

In [30]:
levels, diff, volume = get_evn(COIN, MODE, GRIDS, BUDGET, INTERVAL, STD_NUM, LOWER_STD)

2022-06-21 16:55:40

BTC 현재가(KRW): 27400000.0
호가단위(KRW): 1000
30분봉 40개 변동성: 340495.08
Lower Std: 3
Levels: [26378000, 26579000, 26782000, 26986000, 27192000]
Difference(%): 0.76%
Volume(BTC): 0.00058394
수익률: 0.76%


In [6]:
# 초기 매수 주문
for price in levels:
    ret = upbit.buy_limit_order(f"KRW-{COIN}", price, volume) # 지정된 가격에 매수 주문
    # 에러 발생시 message 출력
    if 'error' in ret.keys():
        print(ret['error']['message'])
    time.sleep(0.1)

while True:
    time.sleep(0.5) # 0.5초마다 업데이트
    
    # 수익률이 STOP_LOSS 보다 작을 경우 모두 시장가로 청산
    current, avg_price, profit = get_profit(COIN) # 현재 손익 가져오기
    if profit < -STOP_LOSS:
        print("**STOP LOSS 실행**")
        print(f"{COIN} 현재가(KRW): {current}")
        print(f"{COIN} 평단가(KRW): {avg_price}")
        print(f"현재 손익: {profit:.2f}%")
        
        # 모든 주문 취소
        cancel_all_order(COIN)
        # 모든 코인 시장가 매도
        sell_all_market(COIN)
        
        print("종료")
        break
    
    # 미체결 주문수가 GRIDS와 다르면 추가 주문
    now_open = len(upbit.get_order(f"KRW-{COIN}")) # 현재 미체결 주문수
    if GRIDS != now_open:
        balance_coin = upbit.get_balance(COIN) # 코인 보유량으로 매수 매도 판단
        last_price = pyupbit.get_current_price(f"KRW-{COIN}") # 최근 체결 가격
        
        if balance_coin != 0: # 보유량이 0이 아니면 최근 체결은 매수
            side = "bid"
            price = last_price + diff # 매도 목표가는 diff 만큼 더한 값
            price = pyupbit.get_tick_size(price)
            ret = upbit.sell_limit_order(f"KRW-{COIN}", price, volume)
            
        else: # 보유량이 0이면 최근 체결은 매도
            side = "ask"
            price = last_price - diff # 매수 목표가는 diff 만큼 뺀 값
            price = pyupbit.get_tick_size(price)
            ret = upbit.buy_limit_order(f"KRW-{COIN}", price, volume)
        
        # 에러 발생시 message 출력
        if 'error' in ret.keys():
            print(ret['error']['message'])
            
        # 최근 체결 정보 표시
        else:
            print("")
            print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            print(f"{side} -> {last_price}KRW  {volume}{COIN} 체결")
            print(f"보유 자산(KRW) {upbit.get_balance('KRW')}")
            print(f"보유 자산({COIN}) {upbit.get_balance(COIN)}")
            
    # print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    # print(f"{COIN} 현재가(KRW):",current)


2022-06-21 15:34:46
bid -> 27251000.0KRW  0.00048757BTC 체결
보유 자산(KRW) 20286.07709083
보유 자산(BTC) 0.0

2022-06-21 15:43:39
ask -> 27340000.0KRW  0.00048757BTC 체결
보유 자산(KRW) 20325.42642768
보유 자산(BTC) 0

2022-06-21 16:07:30
bid -> 27246000.0KRW  0.00048757BTC 체결
보유 자산(KRW) 20325.42642768
보유 자산(BTC) 0.0

2022-06-21 16:15:41
ask -> 27341000.0KRW  0.00048757BTC 체결
보유 자산(KRW) 20361.85131967
보유 자산(BTC) 0

2022-06-21 16:18:27
bid -> 27257000.0KRW  0.00048757BTC 체결
보유 자산(KRW) 20361.85131967
보유 자산(BTC) 0.0

2022-06-21 16:19:21
bid -> 27154000.0KRW  0.00048757BTC 체결
보유 자산(KRW) 20361.85131967
보유 자산(BTC) 0.0

2022-06-21 16:21:29
ask -> 27232000.0KRW  0.00048757BTC 체결
보유 자산(KRW) 20406.61390244
보유 자산(BTC) 0.0

2022-06-21 16:32:26
ask -> 27354000.0KRW  0.00048757BTC 체결
보유 자산(KRW) 20442.05780359
보유 자산(BTC) 0


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\felab\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\felab\AppData\Local\Temp/ipykernel_37952/4029717749.py", line 7, in <module>
    time.sleep(0.2) # 0.2초마다 코인 보유량 가져오기
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\felab\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\felab\appdata\local\programs\python\python37\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed

TypeError: object of type 'NoneType' has no len()

In [65]:
pyupbit.get_current_price(f"KRW-{COIN}")

27338000.0

In [66]:
(current-levels[-1])/levels[-1]

0.009414110089710931

In [67]:
RESET_GRID

5

In [8]:
cancel_all_order(COIN)